### In this file, we consider the PDE system $\mathcal{L} \mathbf{u} = \mathbf{f}$ with zero Dirichlet boundary condition, where
$$
\mathcal{L}=\left[\begin{array}{cc}
1 & -\lambda \Delta \\
\lambda \Delta & 1
\end{array}\right],
\quad
\mathbf{u}=\left[\begin{array}{c} u_1 \\ u_2 \end{array}\right]
\quad
\mathbf{f}=\left[\begin{array}{c} f_1 \\ f_2 \end{array}\right]
$$

In [36]:
import torch
import torch.nn as nn
import numpy as np
import os

In [37]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu_info.name}")
    print(f"GPU memory: {gpu_info.total_memory / 1024**2:.2f} MB")

GPU: NVIDIA GeForce RTX 4090
GPU memory: 24217.31 MB


In [38]:
lambda_ = 0.05
N = 41

n = N - 1
h = 1 / n 
m = n - 1
m2 = m * m
id_m = np.identity(m)
d1= np.identity(m)
d1[0][1] = 1
d1[m - 1][m - 2] = 1
for i in range(0, m):
    d1[i][i] = -2
    if (i >= 1) and (i <= m - 2):
        d1[i][i - 1] = 1
        d1[i][i + 1] = 1
D = np.kron(id_m, d1) + np.kron(d1, id_m)
D = D / (h ** 2)

L = np.zeros((2 * m2, 2 * m2))
L[0: m2, 0: m2] = np.identity(m2)
L[0: m2, m2: 2 * m2] = - lambda_ * D
L[m2: 2 * m2, 0: m2] = lambda_ * D
L[m2: 2 * m2, m2: 2 * m2] = np.identity(m2)

from scipy import sparse
from scipy.sparse.linalg import spsolve
L_sparse = sparse.csr_matrix(L)

In [39]:
f = np.random.rand(5000, 2 * m2)
u = np.zeros_like(f)
for i in range(5000):
    u[i, :] = spsolve(L_sparse, f[i, :])

In [40]:
f, u = torch.tensor(f, dtype=torch.float32), torch.tensor(u, dtype=torch.float32)
f.shape, u.shape

(torch.Size([5000, 3042]), torch.Size([5000, 3042]))

In [41]:
import torch.nn.functional as F

class GreenFun(nn.Module):
    def __init__(self, N):     
        super(GreenFun, self).__init__()
        self.N = N
        self.G_layer = nn.Sequential(nn.Linear(self.N, self.N, bias = False))

    def forward(self, f):   
        return self.G_layer(f)

In [42]:
from torch.utils.data import TensorDataset, DataLoader, random_split

dataset = TensorDataset(f, u)

# 定义训练集和测试集的大小
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# 将数据集按比例分成训练集和测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 获取训练集的数据
train_f = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))])
train_u = torch.stack([train_dataset[i][1] for i in range(len(train_dataset))])

# 获取测试集的数据
test_f = torch.stack([test_dataset[i][0] for i in range(len(test_dataset))])
test_u = torch.stack([test_dataset[i][1] for i in range(len(test_dataset))])

train_f = train_f.to(device)
train_u = train_u.to(device)
test_f = test_f.to(device)
test_u = test_u.to(device)

In [43]:
x = np.linspace(0, 1, N)
y = np.linspace(0, 1, N)
X, Y = np.meshgrid(x, y)

val_U1_np = X * (1 - X) * np.sin(np.pi * Y)
val_U2_np = np.sin(np.pi * X) * Y * (1 - Y)
laplace_u1 = -2 * np.sin(np.pi * Y) - np.pi ** 2 * val_U1_np
laplace_u2 = -2 * np.sin(np.pi * X) - np.pi ** 2 * val_U2_np
val_F1_np = val_U1_np - lambda_ * laplace_u2
val_F2_np = lambda_ * laplace_u1 + val_U2_np

val_f = np.concatenate([val_F1_np[1:-1, 1:-1].flatten(), val_F2_np[1:-1, 1:-1].flatten()])
val_u = np.concatenate([val_U1_np[1:-1, 1:-1].flatten(), val_U2_np[1:-1, 1:-1].flatten()])

val_f = torch.tensor(val_f, dtype=torch.float32).view(1, -1).to(device)
val_u = torch.tensor(val_u, dtype=torch.float32).view(1, -1).to(device)

val_f.shape, val_u.shape

(torch.Size([1, 3042]), torch.Size([1, 3042]))

In [44]:
class EarlyStopping:
    def __init__(self, patience, verbose, delta, path):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.8f} --> {val_loss:.8f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)

In [45]:
net = GreenFun(2 * m2).to(device)
criterion = nn.MSELoss()

import torch.optim.lr_scheduler as lr_scheduler
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience = 1500, factor=0.7, verbose=True)

early_stopping = EarlyStopping(patience = 5000, verbose=False, delta=1e-8, path='net.pth')
num_epochs = 100000

for epoch in range(num_epochs):
    
    net.train()

    optimizer.zero_grad()
    
    output = net(train_f)
    
    loss = criterion(output, train_u)
    loss.backward()
    optimizer.step()

    net.eval()

    with torch.no_grad():
        output_test = net(test_f)
        loss_test = criterion(output_test, test_u)

        if (epoch+1) % 100 == 0:
            output_val = net(val_f)
            loss_val = criterion(output_val, val_u)

            print(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {loss.item():.8f} Testing Loss: {loss_test.item():.8f} Validation loss: {loss_val.item():.8f}")

    # 调整学习率
    scheduler.step(loss_test)

    early_stopping(loss_test, net)
    
    if early_stopping.early_stop:
        print("Early stopping!")
        break

Epoch [100/100000] Training Loss: 0.02363700 Testing Loss: 0.02536865 Validation loss: 0.01578795
Epoch [200/100000] Training Loss: 0.01778465 Testing Loss: 0.02176935 Validation loss: 0.01356084
Epoch [300/100000] Training Loss: 0.01265628 Testing Loss: 0.01830567 Validation loss: 0.01131018
Epoch [400/100000] Training Loss: 0.00881489 Testing Loss: 0.01536019 Validation loss: 0.00931465
Epoch [500/100000] Training Loss: 0.00615272 Testing Loss: 0.01298216 Validation loss: 0.00765400
Epoch [600/100000] Training Loss: 0.00436561 Testing Loss: 0.01109002 Validation loss: 0.00631454
Epoch [700/100000] Training Loss: 0.00316938 Testing Loss: 0.00957761 Validation loss: 0.00524814
Epoch [800/100000] Training Loss: 0.00235715 Testing Loss: 0.00835288 Validation loss: 0.00440090
Epoch [900/100000] Training Loss: 0.00180822 Testing Loss: 0.00735081 Validation loss: 0.00372230
Epoch [1000/100000] Training Loss: 0.00139300 Testing Loss: 0.00650545 Validation loss: 0.00318047
Epoch [1100/100000]

In [46]:
net.load_state_dict(torch.load("net.pth", map_location = device))

<All keys matched successfully>

In [47]:
u1_exact = np.sin(np.pi * X) * Y * (1 - Y)
u2_exact = X * (1 - X) * np.sin(np.pi * Y)
laplace_u1 = -2 * np.sin(np.pi * X) - np.pi ** 2 * u1_exact
laplace_u2 = -2 * np.sin(np.pi * Y) - np.pi ** 2 * u2_exact
f1 = u1_exact - lambda_ * laplace_u2
f2 = lambda_ * laplace_u1 + u2_exact

f = np.concatenate([f1[1:-1, 1:-1].flatten(), f2[1:-1, 1:-1].flatten()])

f_torch = torch.tensor(f, dtype=torch.float32).view(1, -1).to(device)

u_numerical_torch = net(f_torch)
u_numerical = u_numerical_torch.cpu().detach().numpy().flatten()

u1_numerical = np.zeros_like(u1_exact)
u2_numerical = np.zeros_like(u2_exact)
u1_numerical[1:-1, 1:-1] = u_numerical[0: m2].reshape((m, m))
u2_numerical[1:-1, 1:-1] = u_numerical[m2: 2 * m2].reshape((m, m))

In [48]:
def computeErrors(u_exact, u_pre, printOrNot):
    error = u_exact - u_pre
    l2_norm_abs = np.linalg.norm(error, ord=2) / np.sqrt(error.size)
    max_norm_abs = np.max(np.abs(error))
    l2_norm_rel = np.linalg.norm(error, ord=2) / np.linalg.norm(u_exact, ord=2)
    max_norm_rel = np.max(np.abs(error)) / np.max(np.abs(u_exact))  
    
    l2_norm_rel_percent = l2_norm_rel * 100
    max_norm_rel_percent = max_norm_rel * 100
    
    if printOrNot == True:
        print(f"Absolute L2 Norm Error: {l2_norm_abs:.8f}")
        print(f"Absolute Max Norm Error: {max_norm_abs:.8f}")
        print(f"Relative L2 Norm Error: {l2_norm_rel_percent:.6f}%")
        print(f"Relative Max Norm Error: {max_norm_rel_percent:.6f}%")

In [49]:
computeErrors(u1_exact, u1_numerical, True)

Absolute L2 Norm Error: 0.00002744
Absolute Max Norm Error: 0.00022687
Relative L2 Norm Error: 0.021787%
Relative Max Norm Error: 0.090747%


In [50]:
computeErrors(u2_exact, u2_numerical, True)

Absolute L2 Norm Error: 0.00001731
Absolute Max Norm Error: 0.00021974
Relative L2 Norm Error: 0.013743%
Relative Max Norm Error: 0.087896%
